<a href="https://colab.research.google.com/github/HJJunn/DeepLearning---NLP/blob/main/1_KoBERT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT를 이용한 네이버 영화 리뷰 분류하기

In [1]:
!pip install transformers

## 데이터 로드 및 정제

In [2]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [3]:
 urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
 urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7ec90d75c790>)

In [4]:
train_data = pd.read_table("ratings_train.txt")
test_data = pd.read_table("ratings_test.txt")

In [5]:
print(len(train_data))
print(len(test_data))

150000
50000


In [6]:
train_data.drop_duplicates(subset = ['document'], inplace = True)
train_data = train_data.dropna(how = 'any')
print("훈련 데이터 리뷰 수:", len(train_data))

훈련 데이터 리뷰 수: 146182


In [7]:
test_data = test_data.dropna(how = 'any')
print("테스트 데이터 리뷰 수:", len(test_data))

테스트 데이터 리뷰 수: 49997


## 2) BERT의 입력

In [8]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [9]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['보', '##는', '##내', '##내', '그대로', '들어맞', '##는', '예측', '카리스마', '없', '##는', '악역']


In [10]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[2, 1160, 2259, 2369, 2369, 4311, 20657, 2259, 5501, 13132, 1415, 2259, 23713, 3]


In [11]:
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

'[CLS] 보는내내 그대로 들어맞는 예측 카리스마 없는 악역 [SEP]'

In [12]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':', tokenizer.sep_token_id)
print(tokenizer.pad_token, ':', tokenizer.pad_token_id)

[CLS] : 2
[SEP] : 3
[PAD] : 0


In [19]:
#정수 인코딩 with 패딩
max_seq_len = 128

encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고 싶은 영화", max_length= max_seq_len,  padding = 'max_length',truncation=True)

print(encoded_result)
print(len(encoded_result))

[2, 1537, 2534, 2069, 6572, 2259, 3771, 18, 3690, 4530, 1335, 2073, 3771, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
128


In [14]:
#세그먼트 임베딩
print([0] * max_seq_len)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
#어텐션 마스크 임베딩
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시 보고 싶은 영화"))

print(valid_num * [1] + (max_seq_len - valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
#입력 데이터 전체
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    input_ids, attention_masks, token_type_ids, data_labels = [],[],[],[]

    for example, label in tqdm(zip(examples, labels), total = len(examples)):
        input_id =tokenizer.encode(example, max_length = max_seq_len, padding = 'max_length', truncation=True)

        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention_mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token_type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype = int)
    attention_mask = np.array(attention_masks, dtype = int)
    token_type_ids = np.array(token_type_ids, dtype = int)

    data_labels = np.asarray(data_labels, dtype = np.int32)

    return (input_ids, attention_mask, token_type_ids), data_labels

In [21]:
#훈련 데이터
train_X, train_y = convert_examples_to_features(train_data['document'], train_data['label'], max_seq_len= max_seq_len, tokenizer = tokenizer)

100%|██████████| 146182/146182 [01:01<00:00, 2361.95it/s]


In [22]:
#테스트 데이터
test_X, test_y = convert_examples_to_features(test_data['document'], test_data['label'], max_seq_len = max_seq_len, tokenizer = tokenizer)

100%|██████████| 49997/49997 [00:15<00:00, 3127.49it/s]


In [23]:
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]
print('단어에대한정수인코딩:',input_id)
print('어텐션마스크:',attention_mask)
print('세그먼트인코딩:',token_type_id)
print('각인코딩의길이:', len(input_id))
print('정수인코딩복원:',tokenizer.decode(input_id))
print('레이블:',label)

단어에대한정수인코딩: [   2 1376  831 2604   18   18 4229 9801 2075 2203 2182 4243    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
어텐션마스크: [1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
세그먼트인코딩: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

## 3) BERT의 출력 이해하기

In [27]:
model  = TFBertModel.from_pretrained("klue/bert-base", from_pt = True)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [28]:
max_seq_len = 128
input_ids_layer = tf.keras.layers.Input(shape = (max_seq_len,), dtype = tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape = (max_seq_len,), dtype = tf.int32)
token_type_ids_layer =tf.keras.layers.Input(shape = (max_seq_len,), dtype = tf.int32)

outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

In [29]:
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model'")


outputs[0]은 (배치크기, 128, 768)의 크기를 가지는 텐서 -> 768차원의 백터가 128개 있다.

Many-to-Many 문제를 풀 때 사용 ex) 개체명 인식

In [30]:
print(outputs[1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/pooler/dense/Tanh:0', description="created by layer 'tf_bert_model'")


outpus[1]은 (배치크기, 768)의 크기를 가지는 텐서

이는 [CLS]토큰 위치의 출력으로 Many-To-One 문제를 풀 때 사용 ex) 네이버 리뷰, 텍스트 분류 문제

## 4) BERT를 이용한 Many-To-One 모델 만들기

outputs[1] 즉, [CLS] 토큰 위치의 출력을 1개의 뉴런을 가지며 시그모이드 함수가 활성하 함수로 설정된 출력층으로 연결

In [33]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name)
        self.classifier = tf.keras.layers.Dense(1, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.TruncatedNormal(0.02), name = 'classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)
        return prediction

In [34]:
model = TFBertForSequenceClassification("klue/bert-base")
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already u

In [36]:
model.fit(train_X, train_y, epochs = 1, batch_size = 64, validation_split = 0.2)

1828/1828 [==============================] - 2903s 2s/step - loss: 0.2776 - accuracy: 0.8834 - val_loss: 0.2818 - val_accuracy: 0.8904


In [43]:
results = model.evaluate(test_X, test_y, batch_size = 1024)
print('test loss, test acc: ', results)

49/49 [==============================] - 402s 8s/step - loss: 0.2854 - accuracy: 0.8867
test loss, test acc:  [0.285410612821579, 0.8867331743240356]


## 5) 리뷰 예측하기

In [44]:
def sentiment_predict(new_sentence):
    input_id =  tokenizer.encode(new_sentence, max_length = max_seq_len, padding = 'max_length', truncation = True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0]*padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]
    score = model.predict(encoded_input)[0][0]

    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [45]:
sentiment_predict("이 영화 존잼입니다 대박")

1/1 [==============================] - 4s 4s/step
99.35% 확률로 긍정 리뷰입니다.



In [48]:
sentiment_predict("와 개쩐다 정말 세계관 최강자들의 영화다")

1/1 [==============================] - 0s 56ms/step
98.90% 확률로 긍정 리뷰입니다.



In [47]:
sentiment_predict("이딴게 영화냐 ㅉㅉ")

1/1 [==============================] - 0s 58ms/step
95.45% 확률로 부정 리뷰입니다.



In [46]:
sentiment_predict("감독 뭐하는 놈이냐?")

1/1 [==============================] - 0s 56ms/step
96.27% 확률로 부정 리뷰입니다.

